# Preprocessing of revision experiments

This notebook contains the preprocessing steps for the revision experiments. It lists
the expected sessions and checks that they exists on flexilims

In [1]:
sessions = {
    "PZAG17.3a_S20250402": "motor",
    "PZAG17.3a_S20250319": "multidepth",
    "PZAG17.3a_S20250306": "spheretube_5",
    "PZAG17.3a_S20250305": "spheretube_4",
    "PZAG17.3a_S20250303": "spheretube_3",
    "PZAG17.3a_S20250228": "spheretube_2",
    "PZAG17.3a_S20250227": "spheretube_1",
    "PZAG16.3c_S20250401": "motor",
    "PZAG16.3c_S20250317": "multidepth",
    "PZAG16.3c_S20250313": "spheretube_5",
    "PZAG16.3c_S20250310": "spheretube_4",
    "PZAG16.3c_S20250221": "spheretube_3",
    "PZAG16.3c_S20250220": "spheretube_2",
    "PZAG16.3c_S20250219": "spheretube_1",
    "PZAG16.3b_S20250401": "motor",
    "PZAG16.3b_S20250317": "multidepth",
    "PZAG16.3b_S20250313": "spheretube_5",
    "PZAG16.3b_S20250310": "spheretube_4",
    "PZAG16.3b_S20250226": "spheretube_3",
    "PZAG16.3b_S20250225": "spheretube_2",
    "PZAG16.3b_S20250224": "spheretube_1",
    "PZAH17.1e_S20250403": "motor",
    "PZAH17.1e_S20250318": "multidepth",
    "PZAH17.1e_S20250313": "multidepth",
    "PZAH17.1e_S20250311": "spheretube_5",
    "PZAH17.1e_S20250307": "spheretube_4",
    "PZAH17.1e_S20250306": "spheretube_3",
    "PZAH17.1e_S20250305": "spheretube_2",
    "PZAH17.1e_S20250304": "spheretube_1",
}

print(f"{len(sessions)} sessions to analyze")

29 sessions to analyze


In [2]:
import flexiznam as flz

project = "colasa_3d-vision_revisions"
TEMPORARY_FIX = True
flm_sess = flz.get_flexilims_session(project_id=project)
if TEMPORARY_FIX:
    all_sessions = flz.get_entities(datatype="session", flexilims_session=flm_sess)

valid_sessions = dict()
for session, protocol in sessions.items():
    if TEMPORARY_FIX:
        sess = all_sessions[all_sessions["name"] == session]
        if not len(sess):
            sess = None
        else:
            assert len(sess) == 1
            sess = sess.iloc[0]
    else:
        sess = flz.get_entity(name=session, project_id=project, datatype="session")
    if sess is None:
        print(f"Session {session} doesn't exist")
        continue
    valid_sessions[session] = [sess, protocol]
print(f"{len(valid_sessions)}/{len(sessions)} valid sessions to analyze")

29/29 valid sessions to analyze


In [3]:
# Now check that we have the recordings we need
for session_name, (session, protocol) in valid_sessions.items():
    # Get recordings children of the session
    recordings = flz.get_children(
        session.id, children_datatype="recording", flexilims_session=flm_sess
    )
    if not len(recordings):
        print(f"No recordings for session {session_name}")
        continue
    assert (
        "SpheresPermTubeReward" in recordings.protocol.values
    ), f"Session {session_name} doesn't have the Sphere protocol"
    if protocol == "motor":
        assert (
            "SpheresTubeMotor" in recordings.protocol.values
        ), f"Session {session_name} doesn't have the motor protocol"
    elif protocol == "multidepth":
        assert (
            "SpheresPermTubeReward_multidepth" in recordings.protocol.values
        ), f"Session {session_name} doesn't have the multidepth protocol"

In [4]:
# There should be a suite2p dataset for each session
bad_sessions = []
for session_name, (session, protocol) in valid_sessions.items():
    # Get recordings children of the session
    suite2p_dataset = flz.get_entity(
        project_id=project,
        datatype="dataset",
        origin_id=session.id,
        query_key="dataset_type",
        query_value="suite2p_rois",
    )
    if suite2p_dataset is None:
        print(f"Session {session_name} doesn't have a suite2p dataset")
        bad_sessions.append(session_name)

print(f"{len(bad_sessions)} sessions don't have a suite2p dataset")
# remove them from valid_sessions
for session_name in bad_sessions:
    del valid_sessions[session_name]
print(f"{len(valid_sessions)} sessions to analyze after removing bad sessions")

Session PZAH17.1e_S20250318 doesn't have a suite2p dataset
1 sessions don't have a suite2p dataset
28 sessions to analyze after removing bad sessions


In [5]:
# To re-run suite2p see `revisions/run_suite2p.py`

# It is in a separate file as it uses a different conda environment (because of suite2p)

In [ ]:
# To run a single session (for debugging mostly)
from cottage_analysis.pipelines import analysis_pipeline
if True:
    session_name = "PZAG16.3c_S20250317"
    analysis_pipeline.main(project,
        session_name,
        conflicts="overwrite",
        photodiode_protocol=5,
        use_slurm=True,
        run_depth_fit=True,
        run_rf=True,
        run_rsof_fit=True,
        run_plot=True,
        protocol_base= "SpheresPermTubeReward",
    )

------------------------------- 
         Start analysing PZAG16.3c_S20250317   
         -------------------------------
---Start synchronisation...---
Processing recording 1/2
Loading existing monitor frames...
Removing frames in wrong order of frame indices.
Removed 2510 frames including:
0 negative diffs.
2510 duplicates.
Removed 0 frames including:
0 negative diffs.
0 duplicates.
2 frames are not 0.0657 s
ImagingFrames in video: 58122
ImagingFrame triggers: 58123
Processing recording 2/2


/nemo/lab/znamenskiyp/home/users/blota/code/cottage_analysis/cottage_analysis/io_module/harp.py:133: UserWarning: No di_names provided or found in extra_attributes. Using default.
  warnings.warn(


Saving harp messages into npz...


Reading harp messages: 100%|█████████▉| 56.1M/56.1M [00:37<00:00, 1.48Mbits/s]


Harp messages saved.
Recording is 3416 s long.
Frame rate is 143 Hz.
Found 378430 frames out of 432480 render (12.50%, 54050 dropped)
Starting crosscorrelation


...

 11%|█         | 196/1760 [16:52<5:21:13, 12.32s/it]

 11%|█▏        | 200/1760 [17:02<2:42:24,  6.25s/it]

 11%|█▏        | 201/1760 [17:03<2:10:09,  5.01s/it]

 12%|█▏        | 204/1760 [17:04<1:05:25,  2.52s/it]

 12%|█▏        | 206/1760 [17:04<42:34,  1.64s/it]  

In [ ]:
from cottage_analysis.pipelines import pipeline_utils

running = []
sess_to_do = ['PZAG16.3b_S20250313']
if False:
    for session_name in valid_sessions:
        if (sess_to_do is not None )and not (session_name in sess_to_do):
            continue
        protocol = sessions[session_name]
        if protocol.startswith('spheretube'):
            continue
        if session_name in running:
            print(f"Session {session_name} is already running")
            continue
        print(f"Submitting session {session_name} to the pipeline ({protocol})")
        pipeline_utils.sbatch_session(
            project=project,
            session_name=session_name,
            pipeline_filename="run_analysis_pipeline.sh",
            conflicts="overwrite",
            photodiode_protocol=5,
        )



In [ ]:
bad = "PZAH17.1e_S20250318"

sess, protocol = valid_sessions[bad]
sess

In [ ]:
import tifffile

ds = flz.Dataset.from_flexilims(
    id="67ed63e5b99b5006b4e789b5", flexilims_session=flm_sess
)
ds

In [ ]:
img = tifffile.imread(ds.path_full / ds.extra_attributes["tif_files"][0])
img.shape

In [ ]:
flm_sess.delete(ds.id)